In [1]:
import pandas as pd
import requests
import re
import html
import pickle

读取数据

In [2]:
data = pd.read_csv('analysis.csv')
data

,time,link,num,ip
0,2017-05-11 14:09:14,http://www.imooc.com/video/4500,304,218.75.35.226
1,2017-05-11 15:25:05,http://www.imooc.com/video/14623,69,202.96.134.133
2,2017-05-11 07:50:01,http://www.imooc.com/article/17894,115,202.96.134.133
3,2017-05-11 02:46:43,http://www.imooc.com/article/17896,804,218.75.35.226
4,2017-05-11 09:30:25,http://www.imooc.com/article/17893,893,222.129.235.182
...,...,...,...,...
999995,2017-05-11 18:02:34,http://www.imooc.com/article/17898,886,58.32.19.255
999996,2017-05-11 14:52:02,http://www.imooc.com/article/17892,438,202.96.134.133
999997,2017-05-11 03:12:40,http://www.imooc.com/article/17897,662,218.75.35.226
999998,2017-05-11 01:08:40,http://www.imooc.com/article/17899,225,58.32.19.255


提取标题

In [3]:
find = re.compile(r'<title>(.*?)</title>', re.DOTALL)
link_counts = data['link'].value_counts()
title_dict = {}
for i in link_counts.keys():
    title = requests.get(i)
    title = title.content.decode(title.encoding)
    title = re.findall(find, title)[0]
    title = title.replace('\n', '')
    title = html.unescape(title)
    if title == '慕课网':
        title_dict[i] = '该门课程已经下架'
    else:
        title_dict[i] = title
title_dict

{'http://www.imooc.com/video/14540': '课程介绍，轻松愉快之玩转SpringData教程-慕课网',
 'http://www.imooc.com/article/17899': 'Python 对字符串进行处理_慕课手记',
 'http://www.imooc.com/article/17897': 'linux中的用户管理基本命令_慕课手记',
 'http://www.imooc.com/video/4000': '选择合适的数据类型，性能优化之MySQL优化教程-慕课网',
 'http://www.imooc.com/article/17896': '数据库知识点（3）——单行转多列_慕课手记',
 'http://www.imooc.com/video/14704': '.概述-新版，R语言入门与进阶教程-慕课网',
 'http://www.imooc.com/video/14390': '课程的开篇词，机器学习-实现简单神经网络教程-慕课网',
 'http://www.imooc.com/video/14623': 'docker网络，第一个docker化的java应用教程-慕课网',
 'http://www.imooc.com/article/17891': '【花式填坑第1期】高效的前端开发 · Vue.js干货问题合辑_慕课手记',
 'http://www.imooc.com/article/17898': 'SPL迭代器接口（三）—SeekableIterator Interface_慕课手记',
 'http://www.imooc.com/article/17894': 'SPL迭代器接口（一）—Traversable Interface_慕课手记',
 'http://www.imooc.com/video/4600': '该门课程已经下架',
 'http://www.imooc.com/article/17892': 'SPL（Standard PHP Library）迭代器接口简介_慕课手记',
 'http://www.imooc.com/article/17893': '三列布局（为什么要求10个字）_慕课手记',
 'http://www.imooc.com/video/4500'

清除下架视频数据

In [4]:
data = data.drop(data[(data['link'] == 'http://www.imooc.com/video/4600')|(data['link'] == 'http://www.imooc.com/video/4500')].index)
data

,time,link,num,ip
1,2017-05-11 15:25:05,http://www.imooc.com/video/14623,69,202.96.134.133
2,2017-05-11 07:50:01,http://www.imooc.com/article/17894,115,202.96.134.133
3,2017-05-11 02:46:43,http://www.imooc.com/article/17896,804,218.75.35.226
4,2017-05-11 09:30:25,http://www.imooc.com/article/17893,893,222.129.235.182
5,2017-05-11 08:07:35,http://www.imooc.com/article/17891,407,218.75.35.226
...,...,...,...,...
999995,2017-05-11 18:02:34,http://www.imooc.com/article/17898,886,58.32.19.255
999996,2017-05-11 14:52:02,http://www.imooc.com/article/17892,438,202.96.134.133
999997,2017-05-11 03:12:40,http://www.imooc.com/article/17897,662,218.75.35.226
999998,2017-05-11 01:08:40,http://www.imooc.com/article/17899,225,58.32.19.255


添加IP定位

In [5]:
with open('ip_cache', 'rb') as f:
    locals = []
    file = pickle.load(f)
    for i in data['ip']:
        local = file.get(i)
        if local is None:
            break
        else:
            locals.append(local)
if len(locals) == len(data['ip']):
    data['local'] = locals
else:
    raise ValueError
data

,time,link,num,ip,local
1,2017-05-11 15:25:05,http://www.imooc.com/video/14623,69,202.96.134.133,广东省深圳市
2,2017-05-11 07:50:01,http://www.imooc.com/article/17894,115,202.96.134.133,广东省深圳市
3,2017-05-11 02:46:43,http://www.imooc.com/article/17896,804,218.75.35.226,浙江省杭州市
4,2017-05-11 09:30:25,http://www.imooc.com/article/17893,893,222.129.235.182,北京市
5,2017-05-11 08:07:35,http://www.imooc.com/article/17891,407,218.75.35.226,浙江省杭州市
...,...,...,...,...,...
999995,2017-05-11 18:02:34,http://www.imooc.com/article/17898,886,58.32.19.255,上海市
999996,2017-05-11 14:52:02,http://www.imooc.com/article/17892,438,202.96.134.133,广东省深圳市
999997,2017-05-11 03:12:40,http://www.imooc.com/article/17897,662,218.75.35.226,浙江省杭州市
999998,2017-05-11 01:08:40,http://www.imooc.com/article/17899,225,58.32.19.255,上海市


添加经纬度

In [6]:
Coordinates = []
for i in data['local']:
    cache = file.get(i)
    Coordinates.append(cache)
if len(locals) == len(data['ip']):
    data['Coordinates'] = Coordinates
else:
    raise ValueError
data

,time,link,num,ip,local,Coordinates
1,2017-05-11 15:25:05,http://www.imooc.com/video/14623,69,202.96.134.133,广东省深圳市,"114.057868,22.543099"
2,2017-05-11 07:50:01,http://www.imooc.com/article/17894,115,202.96.134.133,广东省深圳市,"114.057868,22.543099"
3,2017-05-11 02:46:43,http://www.imooc.com/article/17896,804,218.75.35.226,浙江省杭州市,"120.209947,30.245853"
4,2017-05-11 09:30:25,http://www.imooc.com/article/17893,893,222.129.235.182,北京市,"116.407526,39.904030"
5,2017-05-11 08:07:35,http://www.imooc.com/article/17891,407,218.75.35.226,浙江省杭州市,"120.209947,30.245853"
...,...,...,...,...,...,...
999995,2017-05-11 18:02:34,http://www.imooc.com/article/17898,886,58.32.19.255,上海市,"121.473701,31.230416"
999996,2017-05-11 14:52:02,http://www.imooc.com/article/17892,438,202.96.134.133,广东省深圳市,"114.057868,22.543099"
999997,2017-05-11 03:12:40,http://www.imooc.com/article/17897,662,218.75.35.226,浙江省杭州市,"120.209947,30.245853"
999998,2017-05-11 01:08:40,http://www.imooc.com/article/17899,225,58.32.19.255,上海市,"121.473701,31.230416"


地区统计

In [7]:
local_counts = data['local'].value_counts()
local_counts

安徽省合肥市    178288
北京市       177874
浙江省杭州市    177787
广东省深圳市    177755
上海市       177429
Name: local, dtype: int64

点击量统计

In [8]:
clicked_counts = {}
link_group = data.groupby('link')
for i, j in link_group:
    clicked_counts[i] = j['num'].sum(axis=0)
clicked_counts

{'http://www.imooc.com/article/17891': 27705482,
 'http://www.imooc.com/article/17892': 27712039,
 'http://www.imooc.com/article/17893': 27786872,
 'http://www.imooc.com/article/17894': 27713974,
 'http://www.imooc.com/article/17895': 27652859,
 'http://www.imooc.com/article/17896': 27957515,
 'http://www.imooc.com/article/17897': 27925993,
 'http://www.imooc.com/article/17898': 27643751,
 'http://www.imooc.com/article/17899': 27950639,
 'http://www.imooc.com/video/14322': 27592386,
 'http://www.imooc.com/video/14390': 27895139,
 'http://www.imooc.com/video/14540': 55454898,
 'http://www.imooc.com/video/14623': 27822312,
 'http://www.imooc.com/video/14704': 27737876,
 'http://www.imooc.com/video/4000': 27847261}

统计量排序

In [9]:
clicked_counts = sorted(clicked_counts.items(), key=lambda x: x[1], reverse=True)
clicked_counts

[('http://www.imooc.com/video/14540', 55454898),
 ('http://www.imooc.com/article/17896', 27957515),
 ('http://www.imooc.com/article/17899', 27950639),
 ('http://www.imooc.com/article/17897', 27925993),
 ('http://www.imooc.com/video/14390', 27895139),
 ('http://www.imooc.com/video/4000', 27847261),
 ('http://www.imooc.com/video/14623', 27822312),
 ('http://www.imooc.com/article/17893', 27786872),
 ('http://www.imooc.com/video/14704', 27737876),
 ('http://www.imooc.com/article/17894', 27713974),
 ('http://www.imooc.com/article/17892', 27712039),
 ('http://www.imooc.com/article/17891', 27705482),
 ('http://www.imooc.com/article/17895', 27652859),
 ('http://www.imooc.com/article/17898', 27643751),
 ('http://www.imooc.com/video/14322', 27592386)]

添加标题

In [10]:
for index, item in enumerate(clicked_counts):
    clicked_counts[index] = list(item)
    title = title_dict.get(item[0])
    clicked_counts[index].insert(1, title)
clicked_counts

[['http://www.imooc.com/video/14540',
  '课程介绍，轻松愉快之玩转SpringData教程-慕课网',
  55454898],
 ['http://www.imooc.com/article/17896', '数据库知识点（3）——单行转多列_慕课手记', 27957515],
 ['http://www.imooc.com/article/17899', 'Python 对字符串进行处理_慕课手记', 27950639],
 ['http://www.imooc.com/article/17897', 'linux中的用户管理基本命令_慕课手记', 27925993],
 ['http://www.imooc.com/video/14390', '课程的开篇词，机器学习-实现简单神经网络教程-慕课网', 27895139],
 ['http://www.imooc.com/video/4000', '选择合适的数据类型，性能优化之MySQL优化教程-慕课网', 27847261],
 ['http://www.imooc.com/video/14623',
  'docker网络，第一个docker化的java应用教程-慕课网',
  27822312],
 ['http://www.imooc.com/article/17893', '三列布局（为什么要求10个字）_慕课手记', 27786872],
 ['http://www.imooc.com/video/14704', '.概述-新版，R语言入门与进阶教程-慕课网', 27737876],
 ['http://www.imooc.com/article/17894',
  'SPL迭代器接口（一）—Traversable Interface_慕课手记',
  27713974],
 ['http://www.imooc.com/article/17892',
  'SPL（Standard PHP Library）迭代器接口简介_慕课手记',
  27712039],
 ['http://www.imooc.com/article/17891',
  '【花式填坑第1期】高效的前端开发 · Vue.js干货问题合辑_慕课手记',
  27705482],
 ['h

时间段统计

In [11]:
data['time'] = data['time'].astype('str')
time_index = pd.date_range(start='2017-05-11 00:00:00', periods=24, freq='H')
time_counts = {}
for i in range(len(time_index) - 1):
    time_count = data[(str(time_index[i]) <= data['time']) & (data['time'] < str(time_index[i + 1]))]
    time_counts[str(time_index[i]) + '-' + str(time_index[i + 1])] = len(time_count)
time_counts

{'2017-05-11 00:00:00-2017-05-11 01:00:00': 38841,
 '2017-05-11 01:00:00-2017-05-11 02:00:00': 38636,
 '2017-05-11 02:00:00-2017-05-11 03:00:00': 38793,
 '2017-05-11 03:00:00-2017-05-11 04:00:00': 39019,
 '2017-05-11 04:00:00-2017-05-11 05:00:00': 38795,
 '2017-05-11 05:00:00-2017-05-11 06:00:00': 39018,
 '2017-05-11 06:00:00-2017-05-11 07:00:00': 38123,
 '2017-05-11 07:00:00-2017-05-11 08:00:00': 38692,
 '2017-05-11 08:00:00-2017-05-11 09:00:00': 38980,
 '2017-05-11 09:00:00-2017-05-11 10:00:00': 38516,
 '2017-05-11 10:00:00-2017-05-11 11:00:00': 38699,
 '2017-05-11 11:00:00-2017-05-11 12:00:00': 38479,
 '2017-05-11 12:00:00-2017-05-11 13:00:00': 38659,
 '2017-05-11 13:00:00-2017-05-11 14:00:00': 38404,
 '2017-05-11 14:00:00-2017-05-11 15:00:00': 38589,
 '2017-05-11 15:00:00-2017-05-11 16:00:00': 38731,
 '2017-05-11 16:00:00-2017-05-11 17:00:00': 38934,
 '2017-05-11 17:00:00-2017-05-11 18:00:00': 38445,
 '2017-05-11 18:00:00-2017-05-11 19:00:00': 38808,
 '2017-05-11 19:00:00-2017-05-1

验证数量

In [12]:
num = 0
for i in time_counts.values():
    num += i
num

889133

数据透视表

In [13]:
data

,time,link,num,ip,local,Coordinates
1,2017-05-11 15:25:05,http://www.imooc.com/video/14623,69,202.96.134.133,广东省深圳市,"114.057868,22.543099"
2,2017-05-11 07:50:01,http://www.imooc.com/article/17894,115,202.96.134.133,广东省深圳市,"114.057868,22.543099"
3,2017-05-11 02:46:43,http://www.imooc.com/article/17896,804,218.75.35.226,浙江省杭州市,"120.209947,30.245853"
4,2017-05-11 09:30:25,http://www.imooc.com/article/17893,893,222.129.235.182,北京市,"116.407526,39.904030"
5,2017-05-11 08:07:35,http://www.imooc.com/article/17891,407,218.75.35.226,浙江省杭州市,"120.209947,30.245853"
...,...,...,...,...,...,...
999995,2017-05-11 18:02:34,http://www.imooc.com/article/17898,886,58.32.19.255,上海市,"121.473701,31.230416"
999996,2017-05-11 14:52:02,http://www.imooc.com/article/17892,438,202.96.134.133,广东省深圳市,"114.057868,22.543099"
999997,2017-05-11 03:12:40,http://www.imooc.com/article/17897,662,218.75.35.226,浙江省杭州市,"120.209947,30.245853"
999998,2017-05-11 01:08:40,http://www.imooc.com/article/17899,225,58.32.19.255,上海市,"121.473701,31.230416"


In [14]:
link_local = data.groupby(['link', 'local'])
for (k1, k2), group in link_local:
    print((k1, k2))
    print(group)

('http://www.imooc.com/article/17891', '上海市')
                       time                                link  num  \
14      2017-05-11 00:38:01  http://www.imooc.com/article/17891  262   
91      2017-05-11 19:56:33  http://www.imooc.com/article/17891   18   
183     2017-05-11 19:01:28  http://www.imooc.com/article/17891  469   
327     2017-05-11 11:42:17  http://www.imooc.com/article/17891   98   
348     2017-05-11 19:14:01  http://www.imooc.com/article/17891  579   
...                     ...                                 ...  ...   
999534  2017-05-11 18:24:32  http://www.imooc.com/article/17891  829   
999535  2017-05-11 06:02:01  http://www.imooc.com/article/17891  603   
999609  2017-05-11 05:55:37  http://www.imooc.com/article/17891  159   
999906  2017-05-11 14:42:51  http://www.imooc.com/article/17891  762   
999994  2017-05-11 12:01:54  http://www.imooc.com/article/17891  354   

                  ip local           Coordinates  
14      58.32.19.255   上海市  121.473701

[11119 rows x 6 columns]
('http://www.imooc.com/article/17899', '上海市')
                       time                                link  num  \
78      2017-05-11 21:32:41  http://www.imooc.com/article/17899   62   
507     2017-05-11 07:33:27  http://www.imooc.com/article/17899  547   
544     2017-05-11 07:27:48  http://www.imooc.com/article/17899  107   
596     2017-05-11 05:41:20  http://www.imooc.com/article/17899  826   
666     2017-05-11 16:30:58  http://www.imooc.com/article/17899  994   
...                     ...                                 ...  ...   
999750  2017-05-11 07:42:57  http://www.imooc.com/article/17899  395   
999902  2017-05-11 12:58:39  http://www.imooc.com/article/17899  668   
999950  2017-05-11 04:42:36  http://www.imooc.com/article/17899  175   
999993  2017-05-11 05:45:53  http://www.imooc.com/article/17899  220   
999998  2017-05-11 01:08:40  http://www.imooc.com/article/17899  225   

                  ip local           Coordinates  
78      58.32

[11135 rows x 6 columns]
('http://www.imooc.com/video/4000', '广东省深圳市')
                       time                             link  num  \
131     2017-05-11 00:00:08  http://www.imooc.com/video/4000   68   
140     2017-05-11 02:57:55  http://www.imooc.com/video/4000  753   
147     2017-05-11 01:14:39  http://www.imooc.com/video/4000  514   
168     2017-05-11 20:26:30  http://www.imooc.com/video/4000  803   
235     2017-05-11 09:09:40  http://www.imooc.com/video/4000  154   
...                     ...                              ...  ...   
999543  2017-05-11 19:33:29  http://www.imooc.com/video/4000  375   
999674  2017-05-11 13:13:39  http://www.imooc.com/video/4000  467   
999747  2017-05-11 10:53:18  http://www.imooc.com/video/4000  508   
999874  2017-05-11 06:06:19  http://www.imooc.com/video/4000  193   
999916  2017-05-11 02:05:43  http://www.imooc.com/video/4000  211   

                    ip   local           Coordinates  
131     202.96.134.133  广东省深圳市  114.057868,22